In [1]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal


In [2]:
def load_dataframe(k):
    with open('multigauss.txt') as f:
        file = f.read()
        array_data = (file.split("\n"))
        counter = 0
        array_data = array_data[4:]
        dimension_1=[]
    
        for data in array_data:
            temp_array = data.strip().split(" ")
            if len(temp_array) >1:
                dimension_1.append([float(temp_array[0]),float(temp_array[1])])
            
    dataframe = np.array([dimension_1])[0]
        
    n = dataframe.shape[0]
    f = dataframe.shape[1]
    
    w = np.random.rand(n,k)
    
    pi = np.random.rand(k,1)
    #μ is a k × f matrix containing the means of each gaussian
    #mu = np.random.rand(k,f)
    #Σ is an f × f × k tensor of covariance matrices. Σ(:, :, i) is the covariance of gaussian i
   
    mu = np.asmatrix(np.random.random((k,f)))
    covariance = np.array([np.asmatrix(np.identity(f)) for i in range(k)])
    
    return dataframe, k, pi, mu, covariance, w, n, f

        

def expectation(X, k, pi, mu, covariance):
    summation_gaussian = 0.0
    data = X.copy()
    n = X.shape[0]
    
    for sample in range(n):
        summation_gaussian = 0
        for gaussian in range(k):
            value_of_gaussian = multivariate_normal.pdf(data[sample, :],mu[gaussian].A1,\
                                                        covariance[gaussian]) * pi[gaussian]
            summation_gaussian += value_of_gaussian
            w[sample, gaussian] = value_of_gaussian
        
        w[sample, :] /= summation_gaussian
    
    return w
    
def maximize_mean(X,k,w,f):
    new_mu_gaussian = []
    n = X.shape[0]
    for gaussian in range(k):
        responsibilities = w[: ,gaussian].sum()
        
        pi[gaussian] = 1/n * responsibilities
        new_mu = np.zeros(f)
        new_covariance = np.zeros((f,f))

        for sample in range(n):
            new_mu += (X[sample, :] * w[sample,gaussian])
            
         #   new_covariance += w[sample, gaussian] * ((X[sample, :] - mu[gaussian, :]).T *\
         #                                           (X[sample, :] - mu[gaussian, :]))
              
        new_mu_gaussian.append(new_mu / responsibilities)
        #covariance[gaussian] = new_covariance / responsibilities
    return new_mu_gaussian

def maximize_covariance(X,k,w,mu,f):
    new_covariance_gaussian = []
    
    for gaussian in range(k):
        responsibilities = w[: ,gaussian].sum()
        
        pi[gaussian] = 1/n * responsibilities
        new_mu = np.zeros(f)
        new_covariance = np.zeros((f,f))

        for sample in range(n):
            #new_mu += (X[sample, :] * w[sample,gaussian])
            
            new_covariance += w[sample, gaussian] * ((X[sample, :] - mu[gaussian, :]).T *\
                                                    (X[sample, :] - mu[gaussian, :]))
        
        new_covariance_gaussian.append(  new_covariance / responsibilities )  
       # mu[gaussian] = new_mu / responsibilities
       # covariance[gaussian] = 
    
    return new_covariance_gaussian
    
    

def maximize_mixtures(k, w, X, pi, mu,n):
    log_likelihood = 0
    for sample in range(n):
        summation_gaussian_value = 0
        for gaussian in range(k):
            summation_gaussian_value += multivariate_normal.pdf(X[sample, :],mu[gaussian, :].A1,\
                                                                         covariance[gaussian, :]) * pi[gaussian]
        
        log_likelihood += np.log(summation_gaussian_value) 
    
    return log_likelihood


def perform_em(X, k, pi, mu, covariance, nIter, w, n, f):
    
    number_iteration = 0
    log_likelihood = 1
    previous_log_likelihood = 0
    
    while((log_likelihood - previous_log_likelihood) > 1e-4):
        number_iteration = number_iteration + 1
        previous_log_likelihood = maximize_mixtures(k, w, X, pi, mu,n)
        w = expectation(X, k, pi, mu, covariance)
        new_means_gaussian = maximize_mean(X,k,w,f)
        new_covariance = maximize_covariance(X,k,w,mu,f)
        
        for gaussian in range(k):
            mu[gaussian] = new_means_gaussian[gaussian]
            covariance[gaussian] = new_covariance[gaussian]
        
        log_likelihood = maximize_mixtures(k, w, X, pi, mu,n)
        print('Iteration %d: log-likelihood is %.6f'%(number_iteration, log_likelihood))
        
    
        

In [3]:
k = 5
dataframe, k, pi, mu, covariance, w, n, f = load_dataframe(k)
perform_em(dataframe, k, pi, mu, covariance, 10, w, n, f)





Iteration 1: log-likelihood is -6160.579772
Iteration 2: log-likelihood is -6099.989034
Iteration 3: log-likelihood is -6017.533444
Iteration 4: log-likelihood is -5876.433048
Iteration 5: log-likelihood is -5732.438933
Iteration 6: log-likelihood is -5633.734562
Iteration 7: log-likelihood is -5578.244178
Iteration 8: log-likelihood is -5546.362068
Iteration 9: log-likelihood is -5532.227796
Iteration 10: log-likelihood is -5527.770580
Iteration 11: log-likelihood is -5526.265606
Iteration 12: log-likelihood is -5525.412193
Iteration 13: log-likelihood is -5524.574611
Iteration 14: log-likelihood is -5523.495067
Iteration 15: log-likelihood is -5521.958606
Iteration 16: log-likelihood is -5519.664319
Iteration 17: log-likelihood is -5516.128044
Iteration 18: log-likelihood is -5510.669115
Iteration 19: log-likelihood is -5502.730594
Iteration 20: log-likelihood is -5492.321597
Iteration 21: log-likelihood is -5480.283724
Iteration 22: log-likelihood is -5469.202238
Iteration 23: log-l